In [210]:
import pandas as pd
import numpy as np
import math
import re

In [211]:
import warnings
warnings.filterwarnings("ignore")

In [212]:
df = pd.read_csv(r'C:\Users\interns\Desktop\20220727_ExecutedOrders.csv')
df[df['Exch'] == 'CBOE']

,Exec Time,ExecID,OrderID,ClientOrderID,Sym,Account,Username,Under,Expiration,Strike,...,Exch,OrderRouter,Side,Status,Price,Qty,TiF,Type,Open/Close,Liquidity
0,09:30:49:459,072620110429258,320040202284346,3200420726000001,SPXW20220726P3880.00,12P52852,sschroer,SPXW,7/26/2022,"3,880.00",...,CBOE,INET:0:1,SELL,Filled,0.75,1,DAY,LIM,OPEN,BT
16,09:40:14:498,320040202290662_f1,320040202290662,3200420726000092,FDX20220729C227.50B1_FDX20220729P227.50B1,12P52852,sschroer,FDX,NaN,0.00,...,CBOE,INET:0:1,"BUY,BUY",Filled,7.90,1,DAY,LIM,CLOSE,BB
21,10:02:05:062,072610111021702,320040202309594,3200420726000177,SPXW20220726P3880.00,12P52852,sschroer,SPXW,7/26/2022,"3,880.00",...,CBOE,INET:0:1,BUY,Filled,0.60,1,DAY,LIM,CLOSE,BT
80,10:29:25:458,320040202329555_f1,320040202329555,3200420726000446,SPX20230421P3175.00B2_SPX20230421P3600.00S1,12P52852,sschroer,SPX,NaN,0.00,...,CBOE,INET:0:1,"SELL,BUY",Filled,11.80,10,DAY,LIM,OPEN,BT
81,10:29:44:058,320040202329767_f1,320040202329767,3200420726000455,SPX20230421P3175.00B2_SPX20230421P3600.00S1,12P52852,sschroer,SPX,NaN,0.00,...,CBOE,INET:0:1,"BUY,SELL",Filled,12.00,10,DAY,LIM,CLOSE,BT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,13:02:12:080,320040202413933_f1,320040202413933,3200420726001216,FFIV20220916C160.00B1_FFIV20220916P160.00B1,12P52852,sschroer,FFIV,NaN,0.00,...,CBOE,INET:0:1,"BUY,BUY",Filled,16.00,1,DAY,LIM,CLOSE,BC
264,13:41:34:285,320040202435721_f1,320040202435721,3200420726001332,PEP20230616C175.00B1_PEP20230616P170.00B1,12P52852,sschroer,PEP,NaN,0.00,...,CBOE,INET:0:1,"BUY,BUY",Filled,24.90,1,DAY,LIM,CLOSE,BB
267,14:32:24:527,320040202455219_f1,320040202455219,3200420726001346,DPZ20220805C390.00B1_DPZ20220805P390.00B1,12P52852,sschroer,DPZ,NaN,0.00,...,CBOE,INET:0:1,"BUY,BUY",Filled,16.08,1,DAY,LIM,CLOSE,BC
270,14:33:42:999,320040202455720_f1,320040202455720,3200420726001359,DPZ20220805C390.00B1_DPZ20220805P390.00B1,12P52852,sschroer,DPZ,NaN,0.00,...,CBOE,INET:0:1,"BUY,BUY",Filled,15.95,1,DAY,LIM,CLOSE,BC


In [213]:
df_penny = pd.read_csv(r'C:\Users\interns\Desktop\penny.csv' , header = None)
df_penny = df_penny[0]
df_penny

0      AAPL
1      ABBV
2      ABNB
3       ABT
4       ACB
       ... 
376     XRT
377       Z
378      ZM
379    ZNGA
380      ZS
Name: 0, Length: 381, dtype: object

In [214]:
exchanges = df['Exch'].unique()
exchanges

# df.iloc[0]['Exch']

array(['CBOE', 'ARCA', 'ISE', 'NBX', 'C2', 'EDGX'], dtype=object)

In [215]:
df['Fee'] = 0
df

,Exec Time,ExecID,OrderID,ClientOrderID,Sym,Account,Username,Under,Expiration,Strike,...,OrderRouter,Side,Status,Price,Qty,TiF,Type,Open/Close,Liquidity,Fee
0,09:30:49:459,072620110429258,320040202284346,3200420726000001,SPXW20220726P3880.00,12P52852,sschroer,SPXW,7/26/2022,"3,880.00",...,INET:0:1,SELL,Filled,0.75,1,DAY,LIM,OPEN,BT,0
1,09:35:51:632,072610110544072,320040202286607,3200420726000010,MSFT20230915P200.00,12P52852,sschroer,MSFT,9/15/2023,200.00,...,INET:0:1,BUY,Filled,11.25,1,DAY,LIM,OPEN,A,0
2,09:35:54:362,072620110544996,320040202286651,3200420726000014,MSFT20230915P200.00,12P52852,sschroer,MSFT,9/15/2023,200.00,...,INET:0:1,SELL,Filled,11.30,1,DAY,LIM,CLOSE,A,0
3,09:35:56:728,072610110545641,320040202286676,3200420726000016,MSFT20230915P200.00,12P52852,sschroer,MSFT,9/15/2023,200.00,...,INET:0:1,BUY,Filled,11.25,1,DAY,LIM,OPEN,A,0
4,09:35:58:520,072620110546400,320040202286695,3200420726000018,MSFT20230915P200.00,12P52852,sschroer,MSFT,9/15/2023,200.00,...,INET:0:1,SELL,Filled,11.30,1,DAY,LIM,CLOSE,A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,15:43:36:055,072620114993949,320040202492797,3200420726001774,AVGO20220729C510.00,12P52852,sschroer,AVGO,7/29/2022,510.00,...,INET:0:1,SELL,Filled,8.30,1,DAY,LIM,CLOSE,A,0
317,15:44:31:127,072620115015315,320040202493504,3200420726001790,NVDA20230915C15.00,12P52852,sschroer,NVDA,9/15/2023,15.00,...,INET:0:1,SELL,Filled,150.65,1,DAY,LIM,OPEN,ASP,0
318,15:44:46:874,072610115019570,320040202493702,3200420726001806,NVDA20230915C15.00,12P52852,sschroer,NVDA,9/15/2023,15.00,...,INET:0:1,BUY,Filled,150.80,1,DAY,LIM,CLOSE,ASP,0
319,15:45:59:386,072610115050834,320040202494668,3200420726001811,NVDA20240621C120.00,12P52852,sschroer,NVDA,6/21/2024,120.00,...,INET:0:1,BUY,Filled,72.65,1,DAY,LIM,OPEN,ASP,0


In [226]:
df_EDGX = pd.read_csv('EDGX_Fees.csv')
df_EDGX = df_EDGX[['Code', 'Fee']]
df_EDGX = df_EDGX.reset_index(drop = True)
df_EDGX

,Code,Fee
0,BC,-0.15
1,BM,-0.35
2,BN,-0.55
3,BO,0.00
4,CA,0.00
5,CT,0.00
6,DC,-0.15
7,DM,-0.35
8,DN,-0.55
9,DO,0.00


In [224]:
df_C2 = pd.read_csv('C2_Fees.csv')
df_C2 = df_C2[['Code', 'Fee']]
df_C2

,Code,Fee
0,BC,0.15
1,BM,0.35
2,BN,0.55
3,BO,0.00
4,CA,0.00
5,CT,0.00
6,DC,0.15
7,DM,0.35
8,DN,0.55
9,DO,0.00


In [ ]:
df_CBOE = pd.read_csv('CBOE_Fees.csv', header = None)
df_CBOE.columns = ['Code', 'Fee']
df_CBOE['Code']

df_CBOE_code = df_CBOE['Code']
df_CBOE[df_CBOE['Code'] == 'BT']

In [242]:
fees = []
test = []
df = pd.read_csv(r'C:\Users\interns\Desktop\20220802_ExecutedOrders.csv')
for i in range(len(df)):

    #AMEX
    if df.iloc[i]['Exch'] == 'AMEX':
        if df_penny.str.contains(df.iloc[i]['Under']).any():
            df.at[i, 'Fee'] = -0.50

        else:
            df.at[i, 'Fee'] = -0.85

    #ARCA
    if df.iloc[i]['Exch'] == 'ARCA':
        #check liquidity code == A
        if df.iloc[i]['Liquidity'] == 'A':
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -0.39
                    df.at[i, 'Fee'] = -0.39
                #non-complex penny
                else:
                    # df.iloc[i]['Fee'] = -0.25
                    df.at[i, 'Fee'] = -0.25
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.75
                #non-complex non-penny
                else:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.75
            # print('hi')

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -000
                    df.at[i, 'Fee'] = -0.00
                #non-complex penny
                else:
                    # df.iloc[i]['Fee'] = -0.50
                    df.at[i, 'Fee'] = -0.50
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.75
                #non-complex non-penny
                else:
                    # df.iloc[i]['Fee'] = -1.10
                    df.at[i, 'Fee'] = -1.10
        
        # else:
        #     df.at[i, 'Fee'] = 'Unknown'
    

    #BOX
    # Who is contra party ?
    if df.iloc[i]['Exch'] == 'BOX':
        df.at[i, 'Fee'] = 'Unknown'
        #liquidity code A


    #BZX
    #??????
    if df.iloc[i]['Exch'] == 'BZX':
        df.at[i, 'Fee'] = 'Unknown'



#check exchange is C2
    if df.iloc[i]['Exch'] == 'C2':
        #check liquidity code == A
        if df.iloc[i]['Liquidity'] == 'A':
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.20
                #non-complex penny
                else:
                    # df.iloc[i]['Fee'] = -0.25
                    df.at[i, 'Fee'] = 0.36
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = 0.60
                #non-complex non-penny
                else:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = 0.65

        if df.iloc[i]['Liquidity'] == 'R' or df.iloc[i]['Liquidity'] == 'ZS':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -000
                    df.at[i, 'Fee'] = -0.48
                #non-complex penny
                else:
                    df.iloc[i]['Fee'] = -0.50
                    df.at[i, 'Fee'] = -0.49
                    
            else:
                #complex non-penny
                # if re.search(df.iloc[i]['Sym'], '_'):
                if '_' in df.iloc[i]['Sym']:
                    # df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.88
                #non-complex non-penny
                else:
                    # df.iloc[i]['Fee'] = -1.10
                    df.at[i, 'Fee'] = -0.93
            
        if df_C2['Code'].str.contains(df.iloc[i]['Liquidity']).any():
            temp_fee_C2 = df_C2[df_C2['Code'] == df.loc[i]['Liquidity']]['Fee']
            df.at[i, 'Fee'] = float(temp_fee_C2)


        else:
            df.at[i, 'Fee'] = 'Unknown'

#CBOE
    if df.iloc[i]['Exch'] == 'CBOE':
        if df_CBOE_code.str.contains(df.iloc[i]['Liquidity']).any():
            temp_fee = df_CBOE[df_CBOE['Code'] == df.loc[i]['Liquidity']]['Fee']
            df.at[i, 'Fee'] = float(temp_fee)
            # df.at[i, 'Fee'] = 'check'
            # print('hi')
        else:
            df.at[i, 'Fee'] = 'Unknown'
    # print('hi')


#EDGX
#check exchange is EDGX
    if df.iloc[i]['Exch'] == 'EDGX':
        if df.iloc[i]['Liquidity'] == 'R':
            print('hi')
        #check for specific liquidity codes
        if df_EDGX['Code'].str.contains(df.iloc[i]['Liquidity']).any():
            temp_fee2 = df_EDGX[df_EDGX['Code'] == df.loc[i]['Liquidity']]
            temp_fee2 = temp_fee2['Fee']
            df.at[i, 'Fee'] = (float(temp_fee2))
            # print(temp_fee2, df.iloc[i]['Liquidity'] )

        #if fee is unknown for certain liqudity codes
        else:
            df.at[i, 'Fee'] = 'Unknown'

#EMLD
#Tier ????
    if df.iloc[i]['Exch'] == 'EMLD':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.20
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.25
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.20
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.25

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.50
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.88
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -1.05

        else:
            df.at[i, 'Fee'] = 'Unknown'


#GEMX
#Tier ????
#complex ?
    if df.iloc[i]['Exch'] == 'GEMX':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = 0.20
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.20
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = 0.25
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.25

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.50
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.99
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.99

        else:
            df.at[i, 'Fee'] = 'Unknown'

#ISE
#Tier ????
#complex ?
    if df.iloc[i]['Exch'] == 'ISE':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = 0.20
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.20
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = 0.25
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.25

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.50
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.99
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.99

        else:
            df.at[i, 'Fee'] = 'Unknown'



#MIAX
#Tier ????
# rebate program ??
    if df.iloc[i]['Exch'] == 'MIAX':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.25
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = -0.21
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.32
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.25

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.25
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.23
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.32
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.30

        else:
            df.at[i, 'Fee'] = 'Unknown'


#MPRL
#Tier ????
#complex ?
    if df.iloc[i]['Exch'] == 'MPRL':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex


                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = 0.25
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.25
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = 0.85
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.85

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex

                #SPY
                if df.iloc[i]['Under'] == 'SPY':
                    df.at[i, 'Fee'] = -0.46

                #QQQ, IWM
                if df.iloc[i]['Under'] == 'QQQ' or df.iloc[i]['Under'] == 'IWM':
                    df.at[i, 'Fee'] = -0.50
                    

                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.50
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.85
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.85

        else:
            df.at[i, 'Fee'] = 'Unknown'


#MRCY/ MRX
#Tier ????
#complex ?
# pim ?
    if df.iloc[i]['Exch'] == 'MRCY' or df.iloc[i]['Exch'] == 'MRX':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex


                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.62
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = -0.47
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -1.05
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.90

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex

                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.65
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.50
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -1.25
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -1.10

        else:
            df.at[i, 'Fee'] = 'Unknown'



#NASD
#Tier ????
#complex ?
    if df.iloc[i]['Exch'] == 'NASD':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex


                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = 0.20
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.20
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = 0.80
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.80

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex

                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.49
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.49
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.85
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.85

        else:
            df.at[i, 'Fee'] = 'Unknown'

    

#PHLX
#penny ?
    if df.iloc[i]['Exch'] == 'PHLX':
        if df.iloc[i]['Liquidity'] == 'A' :
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.10
                #non-complex penny
                else:
                   
                    df.at[i, 'Fee'] = 0.00
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.10
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = 0.00

        if df.iloc[i]['Liquidity'] == 'R':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex

                #complex penny
                if '_' in df.iloc[i]['Sym']:
                
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    
                    df.at[i, 'Fee'] = -0.48
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    
                    df.at[i, 'Fee'] = -0.50
                #non-complex non-penny
                else:
                   
                    df.at[i, 'Fee'] = -0.48

        # else:
        #     df.at[i, 'Fee'] = 'Unknown'


#NBX
    if df.iloc[i]['Exch'] == 'NBX':
        #check liquidity code == A
        if df.iloc[i]['Liquidity'] == 'A' or df.iloc[i]['Liquidity'] == '1':
            #check if penny
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.at[i, 'Fee'] = -0.35
                #non-complex penny
                else:
                    df.iloc[i]['Fee'] = -0.25
                    df.at[i, 'Fee'] = -0.15
                    
            else:
                #complex non-penny
                if '_' in df.iloc[i]['Sym']:
                    df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.30
                #non-complex non-penny
                else:
                    df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -0.15

        if df.iloc[i]['Liquidity'] == 'R' or df.iloc[i]['Liquidity'] == 'ZS':
            if df_penny.str.contains(df.iloc[i]['Under']).any():
                #check if complex
                #complex penny
                if '_' in df.iloc[i]['Sym']:
                    df.iloc[i]['Fee'] = -000
                    df.at[i, 'Fee'] = -0.50
                #non-complex penny
                else:
                    df.iloc[i]['Fee'] = -0.50
                    df.at[i, 'Fee'] = -0.50
                    
            else:
                #complex non-penny
                # if re.search(df.iloc[i]['Sym'], '_'):
                if '_' in df.iloc[i]['Sym']:
                    df.iloc[i]['Fee'] = -0.75
                    df.at[i, 'Fee'] = -1.00
                #non-complex non-penny
                else:
                    df.iloc[i]['Fee'] = -1.10
                    df.at[i, 'Fee'] = -0.85

        else:
            df.at[i, 'Fee'] = 'Unknown'




# print((exchanges))

In [243]:
df_clean = df[['Sym', 'Under', 'Exch', 'Liquidity', 'Fee']]
# df_unknown = df_clean[df_clean['Fee'] == 'Unknown']

# df_clean[df_clean['Exch'] == 'ARCA']
# df_unknown.groupby(by = ['Exch', 'Liquidity']).sum()
# # df_unknown
# df_fee = df['Fee']
# df.Fee.unique()
df2=df_clean.dropna()
# # df2['Fee'].unique()
# for col in df2:
#   print(df2[col].unique())
# df_clean['Fee']
df_clean
df_clean.to_csv(r'C:\Users\interns\Desktop\FindFees2.csv')

In [ ]:
df_arcax = df[df['Exch'] == 'ARCA']
df_arcax[df_arcax['Liquidity'] == 'R']

,Exec Time,Sym,Account,Username,Under,Expiration,Strike,Call/Put,Exch,OrderRouter,...,Price,Qty,TiF,Type,Open/Close,Liquidity,OrderID,ExecID,ClientOrderID,Fee
97,15:12:15:267,GOOGL20240119P85.00,26P531527,pgolec,GOOGL,1/19/2024,85,P,ARCA,INET:0:1,...,5,1.0,DAY,LIM,OPEN,R,3.200400e+14,8.02201E+13,3.200120e+15,-1.1
304,15:10:32:259,GOOGL20240119P50.00,26P531527,pgolec,GOOGL,1/19/2024,50,P,ARCA,INET:0:1,...,0.83,3.0,DAY,LIM,CLOSE,R,3.200400e+14,8.02201E+13,3.200120e+15,-1.1
370,15:08:52:047,TMO20220909P520.00,17P52867,mmoreno,TMO,9/9/2022,520,P,ARCA,INET:0:1,...,3.2,1.0,DAY,LIM,CLOSE,R,3.200400e+14,8.02101E+13,3.205320e+15,-1.1
440,15:06:57:304,NVDA20230616C240.00,21P530324,sumintern,NVDA,6/16/2023,240,C,ARCA,INET:0:1,...,18,1.0,DAY,LIM,CLOSE,R,3.200400e+14,8.02101E+13,3.204320e+15,-0.5
451,15:06:38:620,NVDA20230915P50.00,26P531527,pgolec,NVDA,9/15/2023,50,P,ARCA,INET:0:1,...,1.11,10.0,DAY,LIM,CLOSE,R,3.200400e+14,8.02101E+13,3.200120e+15,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21854,09:55:30:648,NVAX,14P52874,dbegin,NVAX,NaN,0,NaN,ARCA,INET:0:1,...,55.13,100.0,DAY,LIM,OPEN,R,3.200400e+14,8.02101E+13,3.203620e+15,-0.5
22296,09:50:34:153,TDOC,9P526109,dzawko,TDOC,NaN,0,NaN,ARCA,INET:0:1,...,35.74,50.0,DAY,LIM,OPEN,R,3.200400e+14,8.02101E+13,3.201320e+15,-0.5
23058,09:43:58:026,NVDA20230915C265.00,19P529523,theitz,NVDA,9/15/2023,265,C,ARCA,INET:0:1,...,16.05,1.0,DAY,LIM,OPEN,R,3.200400e+14,8.02101E+13,3.202020e+15,-0.5
23197,09:42:04:242,NVDA20240119P180.00,25P531426,jwood,NVDA,1/19/2024,180,P,ARCA,INET:0:1,...,36.5,1.0,DAY,LIM,CLOSE,R,3.200400e+14,8.02201E+13,3.202320e+15,-0.5


In [ ]:
df[['Sym', 'Under', 'Exch', 'Liquidity', 'Fee']]

In [ ]:
df_EDGX[df_EDGX['Code'] == 'BN']

,Code,Fee
2,BN,-0.55


In [ ]:
df_EDGX[df_EDGX['Code'] == 'BN']['Fee']

2   -0.55
Name: Fee, dtype: float64

In [ ]:
df.Fee.unique()

array([0.0, 'Unknown', -1.1, -0.88], dtype=object)

In [ ]:
df_unknown = df[df['Fee'] == 'Unknown'][['Sym', 'Under', 'Exch', 'Liquidity', 'Fee']]
df_unknown[df_unknown['Exch'] == 'ARCA']['Liquidity'].unique()

array(['ASP'], dtype=object)

In [ ]:
df[df['Exch'] == 'ARCA']['Fee'].unique()

array(['Unknown', -1.1], dtype=object)

In [234]:
z = df_EDGX[df_EDGX['Code'] == 'ZT']

# float(z)
z = z['Fee']
float(z)

-0.48